In [0]:
# Install Java, Spark, and Findspark
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/spark-2.4.5/spark-2.4.5-bin-hadoop2.7.tgz
!tar xf spark-2.4.5-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.5-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

In [7]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2020-05-18 03:22:11--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar.1’

postgresql-42.2.9.j 100%[===================>] 892.61K  4.78MB/s    in 0.2s    

2020-05-18 03:22:12 (4.78 MB/s) - ‘postgresql-42.2.9.jar.1’ saved [914037/914037]



In [0]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CloudETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [10]:
#load video game reviews data from s3.amazonaws.com
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Software_v1_00.tsv.gz"

spark.sparkContext.addFile(url)

software_df = spark.read.option("header","true").csv(SparkFiles.get("amazon_reviews_us_Software_v1_00.tsv.gz"), inferSchema=True, sep="\t")

software_df.show(10)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|        review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|         US|   42605767|R3EFW2STIYIY0I|B00MUTIDKI|     248732228|McAfee 2015 Inter...|        Software|          1|            2|          2|   N|                Y|I was very disapp...|I was very disapp...|2015-08-31 00:00:00|
|         US|   51771800|R12NR0R5A9F7FT|B00EPACNUG|     531462352|Hallmark Card Stu...| 

## Drop duplicates and incomplete rows

In [11]:
#drop duplicates and incomplete rows
print(software_df.count())
software_df = software_df.dropna()
print(software_df.count())
software_df = software_df.dropDuplicates()
#show total counts of reviews after dropping duplicates and incomplete rows
print(software_df.count())

341931
341913
341913


## Examine the schema

In [12]:
software_df.printSchema()

root
 |-- marketplace: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- product_title: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)
 |-- review_headline: string (nullable = true)
 |-- review_body: string (nullable = true)
 |-- review_date: timestamp (nullable = true)



## Create a new DataFrame for vine reviews info

In [13]:
vine_reviews = software_df.select(["review_id","star_rating","helpful_votes","total_votes","vine"])
vine_reviews.show(10)

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
|R2C2BVCJT25XGE|          5|            0|          0|   N|
| R15FOVYL149WE|          1|            1|          2|   N|
|R1GAZQX4IY4XXI|          5|            1|          1|   N|
| RIETOLYTNWF90|          3|            0|          0|   N|
|R20CXVX1V4BZDW|          5|            0|          0|   N|
|R12IN3O6THHVUX|          4|            0|          0|   N|
|R3RG7V3APCIF9A|          5|            0|          0|   N|
|R3VY6PQJZQE25Y|          5|            0|          1|   N|
| R91T2APEVHHF2|          5|            0|          0|   N|
|R16TJZM1ED2QOU|          1|            1|          1|   N|
+--------------+-----------+-------------+-----------+----+
only showing top 10 rows



In [14]:
#select vine reviews
vine_reviews_yes = vine_reviews.filter(vine_reviews["vine"]=="Y")
#show the total count of vine reviews
print(vine_reviews_yes.count())
vine_reviews_yes.show(10)

10415
+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
|R1P0EO605W4X4E|          4|            1|          2|   Y|
|R2C6QMNTMOEWO1|          5|            2|         15|   Y|
| RKS9UBOIVBWAS|          4|            1|          2|   Y|
| R3P0YB3905OX1|          5|            2|          2|   Y|
|R1PT5SI2DVKACS|          5|            1|          1|   Y|
|R1H43S0F8CD17E|          1|            5|          5|   Y|
|R2L43434P0K0VT|          5|            0|          0|   Y|
|R35ZJHQME3IR49|          5|           49|         51|   Y|
|R1LJO7L4Y1N452|          5|            5|          8|   Y|
|R1XI4RP8200IMK|          1|           44|         48|   Y|
+--------------+-----------+-------------+-----------+----+
only showing top 10 rows



## Write DataFrame to RDS

In [0]:
#replace <your ...> with your database connection information
mode="append"
jdbc_url = "jdbc:postgresql://<your connection string>:5432/<your database-name>"
config = {"user":"<your database user>",
          "password": "<your database password>",
          "driver":"org.postgresql.Driver"}

In [0]:
vine_reviews_yes.write.jdbc(url=jdbc_url, table='software_vine_reviews', mode=mode, properties=config)